# 测试通过嵌入检索的成功率 - 基于gte-qwen2-1.5b-instruct-embed

初步结论：

- k=1时，达到69%，是合格的水平
- k=2时，达到84%
- k=3时，达到89%
- k=5时，91.7%

效果很满意。

达到我的预期，嵌入的召回率在70%到90%之间。

## 准备

In [11]:
%%time

INDEX_PATH="retrieve-index2-2"
DATA_PATH="retrieve-data2-2"

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [12]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 5.03 ms, sys: 8.19 ms, total: 13.2 ms
Wall time: 123 ms


In [13]:
%%time


import json

file_path = './news-data-from-shen.json'
with open(file_path, 'r', encoding='utf-8') as file:
    test_data = json.load(file)

len(test_data)

CPU times: user 1.32 ms, sys: 0 ns, total: 1.32 ms
Wall time: 1.21 ms


205

In [17]:
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)

In [18]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):

    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Mobile Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if 'finance.china.com.cn' in url:
        response.encoding = 'utf-8'
        
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10.3 µs


In [ ]:
%%time

!rm -rf $DATA_PATH
!mkdir -p $DATA_PATH

import json

for news in test_data:
    data=get_news_data(news['url'])
    data['url']=news['url']
    file_path = f'./{DATA_PATH}/{data["title"]}.json'
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [19]:
%%time

import glob

# 指定目录路径
directory = f'./{DATA_PATH}'

# 使用 glob 模块匹配所有的 .json 文件
json_files = glob.glob(f"{directory}/*.json")

# 输出文件数量
print(f"在 {directory} 目录下有 {len(json_files)} 个 .json 文件。")


在 ./retrieve-data2-2 目录下有 200 个 .json 文件。
CPU times: user 0 ns, sys: 1.77 ms, total: 1.77 ms
Wall time: 1.5 ms


In [20]:
%%time

from llama_index.core import(
    Document
)

def data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.15 µs


In [21]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir=f"./{DATA_PATH}").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

CPU times: user 54.8 ms, sys: 4.31 ms, total: 59.1 ms
Wall time: 3.2 s


In [22]:
%%time

import json

docs=[]
for document in documents:
    news_data=json.loads(document.text) # documents[0].text
    docs.append(data2doc(news_data))

documents=docs

len(docs)

CPU times: user 17 ms, sys: 0 ns, total: 17 ms
Wall time: 16 ms


200

In [23]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 0 ns, sys: 49 µs, total: 49 µs
Wall time: 51.3 µs


In [24]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_embedding("rjmalagon/gte-qwen2-1.5b-instruct-embed-f16")

CPU times: user 2.29 ms, sys: 0 ns, total: 2.29 ms
Wall time: 2.07 ms


In [25]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/200 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/200 [00:00<?, ?it/s]

current doc id: 1至5月份规模以上工业企业利润保持增长
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.document_summary.base:> Generated summary for doc 1至5月份规模以上工业企业利润保持增长: The provided text discusses the performance of the Chinese industrial sector in terms of profit generation for the period between January to May 2024. It highlights several key points:

1. **Growth in Industrial Profits**: The national规模以上工业企业 (industrial enterprises above a certain size) saw an increase in profits, with total profits reaching 27543.8 billion yuan during this period, marking a growth rate of 3.4% compared to the same period last year.

2. **Recovery in Industrial Production**: Macro policies are said to have had a positive impact on market demand and industrial production, leading to a stable recovery in profits for these enterprises.

3. **Industry-Specific Performance**:
   - **Agriculture, Water Supply, and Power Generation**: This sector s

Generating embeddings:   0%|          | 0/200 [00:00<?, ?it/s]

CPU times: user 5.31 s, sys: 220 ms, total: 5.53 s
Wall time: 43min 51s


In [26]:
%%time

doc_summary_index.storage_context.persist(INDEX_PATH)

CPU times: user 892 ms, sys: 15.6 ms, total: 908 ms
Wall time: 907 ms


In [26]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=INDEX_PATH)
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 102 ms


## 检索

### 简单测试

In [27]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve(test_data[0]["question"])

CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 129 ms


In [28]:
retrieved_nodes[0].metadata

{'title': '冯超：普京访越，透视出的国家相处之道应该是什么？',
 'publish_time': '2024-06-23 08:48:43',
 'author': '小婷',
 'url': 'https://www.guancha.cn/fengchao/2024_06_23_738933_s.shtml',
 'images': ['https://i.guancha.cn/news/dfic/2024/06/21/20240621150313664.jpg',
  'https://i.guancha.cn/news/dfic/2024/06/21/20240621150436659.jpg',
  'https://i.guancha.cn/shiping-banner.jpg']}

In [32]:
retrieved_nodes[0].metadata['url']==test_data[0]["url"]

False

### k=1

In [28]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=1,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        # print(news["question"])
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results

CPU times: user 2.61 s, sys: 10.8 ms, total: 2.63 s
Wall time: 22.4 s


[True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 

In [29]:
results.count(True) / len(results)

0.6926829268292682

### k=2

In [30]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=2,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 2.67 s, sys: 16.3 ms, total: 2.69 s
Wall time: 22.5 s


0.8439024390243902

### k=3

In [31]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 2.71 s, sys: 32.5 ms, total: 2.74 s
Wall time: 22.5 s


0.8926829268292683

### k=5

In [32]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=5,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 2.81 s, sys: 37.2 ms, total: 2.85 s
Wall time: 22.7 s


0.9170731707317074

### k=6

In [33]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=6,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 2.88 s, sys: 28.7 ms, total: 2.91 s
Wall time: 22.9 s


0.9219512195121952

### k=10

In [34]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=10,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 3.1 s, sys: 55 ms, total: 3.16 s
Wall time: 23 s


0.9414634146341463

### k=100

In [35]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=len(test_data)/2,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 10 s, sys: 46 ms, total: 10.1 s
Wall time: 24.6 s


0.9853658536585366